# 전처리

In [1]:
import numpy as np
import pandas as pd
import re
from bs4 import BeautifulSoup
from konlpy.tag import Okt

In [2]:
# 데이터 불러오기
# 도서 데이터
df= pd.read_excel('경기대 인기도서 50권.xlsx')

# 2 특수문자 및 태그 제거
# 3 영어 및 숫자 제거

In [3]:
# 특수 문자, 영어, 숫자, 개행 문자 제거 함수 정의
def remove_special_chars(text):
    if isinstance(text, str):
        soup = BeautifulSoup(text, "html.parser")
        cleaned_text = soup.get_text()
        cleaned_text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣ\s]', '', cleaned_text)
        cleaned_text = re.sub('[\r\n]+', ' ', cleaned_text)
        return cleaned_text
    else:
        return text

# 대출 횟수를 제외한 컬럼에 대해서 수행
# 특수 문자, 영어, 숫자, 개행 문자 제거할 컬럼들 리스트
columns_to_clean = ['도서명', '저자', '출판사', '발행연도', '카테고리', '저자소개', '책소개', '목차', '출판사서평', '추천사']

# 각 컬럼에 특수 문자, 영어, 숫자, 개행 문자 제거 적용
for column in columns_to_clean:
    df[column] = df[column].apply(remove_special_chars)

# 4 토큰화 및 형태소 분석

In [4]:
# Okt
tokenizer = Okt()

# Tokenization and morphological analysis 함수
def tokenize_and_analyze(text):
    if isinstance(text, str):
        tokens = tokenizer.morphs(text)  # Morphological analysis
        return tokens
    else:
        return []

# 토큰화 및 형태소 분석을 적용할 컬럼들 리스트
columns_to_tokenize = ['도서명', '저자', '출판사', '저자소개', '책소개', '목차', '출판사서평', '추천사']

# 각 컬럼에 토큰화 및 형태소 분석 적용
for column in columns_to_tokenize:
    df[column] = df[column].apply(tokenize_and_analyze)


# 5  텍스트 임베딩

In [5]:
from gensim.models import Word2Vec

In [6]:
columns_to_embedding = ['저자소개', '책소개', '출판사서평', '추천사']

author_model = Word2Vec(df[columns_to_embedding[0]], min_count=1)
bood_model = Word2Vec(df[columns_to_embedding[1]], min_count=1)
publisher_model = Word2Vec(df[columns_to_embedding[2]], min_count=1)
recommend_model = Word2Vec(df[columns_to_embedding[3]], min_count=1)

# 6 불용어 처리

In [7]:
import nltk
from nltk.corpus import stopwords

In [8]:
# 불용어 제거 함수
def remove_stopwords(tokens):
  # 불용어 리스트
  with open('stopwords.txt', 'r', encoding='utf-8') as file:
        korean_stopwords = [line.strip() for line in file]
  filtered_tokens = [token for token in tokens if token not in korean_stopwords]
  return filtered_tokens

# 불용어 제거를 적용할 컬럼
columns_to_clean = ['저자소개', '책소개', '목차','서평+추천사']

# 컬럼 별로 불용어 제거
for column in columns_to_tokenize:
    df[column] = df[column].apply(remove_stopwords)

# 7 맞춤법 및 띄어쓰기

In [9]:
from hanspell import spell_checker

In [10]:
def preprocess_text(text):
    spelled_text = spell_checker.check(text)
    checked_text = spelled_text.checked
    return checked_text

In [11]:
# 전처리할 열 목록
columns_to_preprocess = ['저자소개', '책소개', '출판사서평', '추천사']

In [12]:
# 각 열에 대한 전처리 적용
for column in columns_to_preprocess:
    df[column] = df[column].apply(lambda x: preprocess_text(x) if isinstance(x, str) else x)

In [13]:
df[columns_to_preprocess]

,저자소개,책소개,출판사서평,추천사
0,"[부산, 태어났다, 부산, 대, 재료공학, 공부, 하고, 반도체, 엔지니어, 했다,...","[여기는, 잠들어야만, 입장, 할, 수, 있는, 달러, 구트, 꿈, 백화점, 입니다...","[텀블벅, 펀딩, 달성, 출, 간, 주, 연속, 종합, 베스트셀러, 위, 독자, 별...",[]
1,"[화만, 화, 소설, 넘나, 들며, 온갖, 이야기, 써, 나가는, 전, 천후, 스토...","[최고, 재미, 감동, 선사, 하는, 힐링, 소설, 원조, 주요, 서점, 종합, 베...","[원, 플러스, 원, 기쁨, 삼각김밥, 모양, 슬픔, 만, 원, 번의, 폭소, 터지...","[서울역, 홈리스, 지내면서도, 안위, 보다는, 지갑, 잃어버린, 낯선, 부인, 안..."
2,"[어둠의군주, 하이, 마, 태양, 왕, 천년, 마법사, 달빛조각사, 새벽, 여행자,...","[첫, 선, 보인, 온라인과, 오프라인, 넘나, 들며, 다양한, 방식, 대한민국, ...",[],[]
3,"[전, 세계, 뜨거운, 관심, 사랑, 받고, 있는, 한국, 계, 미국인, 소설가, ...","[역사, 는, 버렸지만, 상관없다, 역사, 외면, 당, 한, 재일조선인, 가족, 대...","[회복, 연민, 대한, 강력한, 이야기, 버락, 오바마, 미국, 전, 대통령, 전,...","[올해, 최고, 책, 록산, 게이, 작가, 세기, 견뎌내고, 번영, 이룬, 재일한국..."
4,"[미국, 작가, 가난, 때문, 오레곤, 숙모, 집, 고등학교, 졸업, 한, 허버트,...","[역사상, 가장, 많이, 팔린, 이자, 독자, 뽑은, 역대, 최고, 인, 프랭크, ...","[듄, 견줄, 수, 있는, 건, 반지, 제왕, 외, 에는, 없다, 아서, 클라크, ...","[듄, 견줄, 수, 있는, 건, 반지, 제왕, 외, 에는, 없다, 아서, 클라크, ..."
5,"[영국, 요크셔, 출신, 동화작가, 겸, 소설가, 다, 출, 간, 한, 첫, 소설,...","[밤, 시, 죽기, 전, 에만, 열리는, 마법, 도서관, 인생, 두, 번째, 기회,...","[영국, 아마존, 종합, 베스트셀러, 위, 미국, 아마존, 뉴욕타임스, 선데이타임스...",[]
6,"[소설가, 년생, 포스텍, 화학, 전공, 하고, 생화학, 석사, 학위, 받았, 다년...","[김, 초엽, 첫, 장편소설, 간절히, 기다려, 온, 이야기, 이미, 폭넓은, 독자...","[악마, 식물, 내, 정원, 자라고, 있는데, 거, 멸망, 징조, 아니야, 덩굴식물...","[간절히, 원, 해온, 은, 울, 고, 난, 뒤, 에야, 알, 게, 되는, 걸까, ..."
7,"[이후, 한국, 정의, 열풍, 불러일으켰다, 세, 최연, 소, 하버드, 대학교, 교...","[마이클, 샌델, 만의, 신간, 공정하다는, 착각, 출, 간, 샌델, 기울어진, 사...","[능력, 주의, 는, 공정하게, 작동, 하는가, 공정함, 정의, 란, 공식, 은, ...","[역시, 마이클, 샌델답다, 에는, 시대, 가장, 예민한, 이슈, 수술, 메스, 대..."
8,"[부산, 태어났다, 부산, 대, 재료공학, 공부, 하고, 반도체, 엔지니어, 했다,...","[만, 독자, 사로잡은, 달러, 구트, 꿈, 백화점, 두, 번째, 이야기, 어느덧,...","[내내, 베스트셀러, 위, 독자, 뽑은, 최고, 힐링, 판타지, 잠들어야만, 입장,...",[]
9,"[대학, 컴퓨터공학, 전공, 하고, 소프트웨어, 개발자, 했다, 번의, 입사, 퇴사...","[받아들여지고, 있다는, 느낌, 이렇게, 슬프고도, 좋을, 줄, 이야, 브런치, 북...","[먼저, 읽은, 독자, 요청, 종이, 책, 다시, 태어난, 소설, 출, 간, 전자책...","[내, 맘대로, 올해, 책, 모자라지도, 하지도, 않은, 적당한, 온도, 따뜻함과,..."


In [14]:
df

,순위,도서명,저자,출판사,발행연도,대출횟수,카테고리,저자소개,책소개,목차,출판사서평,추천사
0,1,"[달러, 구트, 꿈, 백화점, 주문, 하신, 꿈, 은, 매진, 입니다, 이미, 장편소설]",[이미],"[팩토리, 나인, 쌤앤, 파커, 스]",,78,국내도서 소설시희곡 장르소설 판타지 국내도서 소설시희곡 한국소설 한국 장편소설 국내...,"[부산, 태어났다, 부산, 대, 재료공학, 공부, 하고, 반도체, 엔지니어, 했다,...","[여기는, 잠들어야만, 입장, 할, 수, 있는, 달러, 구트, 꿈, 백화점, 입니다...","[작가, 말, 프롤로그, 번째, 제자, 유서, 깊은, 가게, 주문, 하신, 꿈, 은...","[텀블벅, 펀딩, 달성, 출, 간, 주, 연속, 종합, 베스트셀러, 위, 독자, 별...",[]
1,2,"[불편한, 편의점, 김호연, 장편소설]",[김호연],"[나무, 옆, 의자]",,78,국내도서 소설시희곡 한국소설 한국 장편소설 국내도서 올해의 책 년 올해의 책 후보...,"[화만, 화, 소설, 넘나, 들며, 온갖, 이야기, 써, 나가는, 전, 천후, 스토...","[최고, 재미, 감동, 선사, 하는, 힐링, 소설, 원조, 주요, 서점, 종합, 베...","[산, 해, 진미, 도시락, 제이, 에스, 오브, 제이, 에스, 삼각김밥, 용도, ...","[원, 플러스, 원, 기쁨, 삼각김밥, 모양, 슬픔, 만, 원, 번의, 폭소, 터지...","[서울역, 홈리스, 지내면서도, 안위, 보다는, 지갑, 잃어버린, 낯선, 부인, 안..."
2,3,"[달빛조각사, 남희성, 게임, 판타지, 소설]",[남희성],"[로크, 미디어]",2014,73,국내도서 소설시희곡 장르소설 판타지,"[어둠의군주, 하이, 마, 태양, 왕, 천년, 마법사, 달빛조각사, 새벽, 여행자,...","[첫, 선, 보인, 온라인과, 오프라인, 넘나, 들며, 다양한, 방식, 대한민국, ...","[다크, 게이머, 탄생, \t\t, 독종, 등장, \t\t, 교관, 의뢰, \t\t...",[],[]
3,4,"[파친코, 이민, 진, 장편소설]",[],"[문학사, 상]",,57,국내도서 소설시희곡 영미소설 영미 장편소설 국내도서 미디어 추천 한겨레 국내도서 ...,"[전, 세계, 뜨거운, 관심, 사랑, 받고, 있는, 한국, 계, 미국인, 소설가, ...","[역사, 는, 버렸지만, 상관없다, 역사, 외면, 당, 한, 재일조선인, 가족, 대...",[],"[회복, 연민, 대한, 강력한, 이야기, 버락, 오바마, 미국, 전, 대통령, 전,...","[올해, 최고, 책, 록산, 게이, 작가, 세기, 견뎌내고, 번영, 이룬, 재일한국..."
4,5,"[듄, 개정판]",[],[황금가지],,55,국내도서 소설시희곡 장르소설 국내도서 소설시희곡 영미소설 영미 장편소설,"[미국, 작가, 가난, 때문, 오레곤, 숙모, 집, 고등학교, 졸업, 한, 허버트,...","[역사상, 가장, 많이, 팔린, 이자, 독자, 뽑은, 역대, 최고, 인, 프랭크, ...","[듄, 부록, 듄, 생태계, 부록, 듄, 종교, 부록, 베네, 게, 세, 리트, 의...","[듄, 견줄, 수, 있는, 건, 반지, 제왕, 외, 에는, 없다, 아서, 클라크, ...","[듄, 견줄, 수, 있는, 건, 반지, 제왕, 외, 에는, 없다, 아서, 클라크, ..."
5,6,"[미드나잇, 라이브러리]",[],"[인, 플루, 엔, 셜]",,54,국내도서 소설시희곡 영미소설 영미 장편소설 국내도서 올해의 책 년 올해의 책 후보...,"[영국, 요크셔, 출신, 동화작가, 겸, 소설가, 다, 출, 간, 한, 첫, 소설,...","[밤, 시, 죽기, 전, 에만, 열리는, 마법, 도서관, 인생, 두, 번째, 기회,...",[],"[영국, 아마존, 종합, 베스트셀러, 위, 미국, 아마존, 뉴욕타임스, 선데이타임스...",[]
6,7,"[지구, 끝, 온실, 김, 초엽, 장편소설]","[김, 초엽]","[자이언트, 북스]",,48,국내도서 소설시희곡 장르소설 국내도서 소설시희곡 한국소설 한국 장편소설 국내도서 ...,"[소설가, 년생, 포스텍, 화학, 전공, 하고, 생화학, 석사, 학위, 받았, 다년...","[김, 초엽, 첫, 장편소설, 간절히, 기다려, 온, 이야기, 이미, 폭넓은, 독자...","[프롤로그, 장, 모스, 바나, 장, 프림, 빌리지, 장, 지구, 끝, 온실, 작가...","[악마, 식물, 내, 정원, 자라고, 있는데, 거, 멸망, 징조, 아니야, 덩굴식물...","[간절히, 원, 해온, 은, 울, 고, 난, 뒤, 에야, 알, 게, 되는, 걸까, ..."
7,8,"[공정하다는, 착각, 능력, 주의, 는, 같은, 기회, 제공, 하는가]",[],"[와이즈, 베리]",,43,국내도서 사회 정치 사회학 사회사사회문화 국내도서 인문 윤리학 국내도서 미디어 추천...,"[이후, 한국, 정의, 열풍, 불러일으켰다, 세, 최연, 소, 하버드, 대학교, 교...","[마이클, 샌델, 만의, 신간, 공정하다는, 착각, 출, 간, 샌델, 기울어진, 사...","[서론, 대학, 입시, 능력, 주의, 입시, 윤리, 능력, 지표, 따내기, 승자, ...","[능력, 주의, 는, 공정하게, 작동, 하는가, 공정함, 정의, 란, 공식, 은, ...","[역시, 마이클, 샌델답다, 에는, 시대, 가장, 예민한, 이슈, 수술, 메스, 대..."
8,9,"[달러, 구트, 꿈, 백화점, 이미, 장편소설, 단골, 손님, 찾습니다]",[이미],"[팩토리, 나인, 쌤앤, 파커, 스]",,42,국내도서 소설시희곡 장르소설 판타지 국내도서 소설시희곡 한국소설 한국 장편소설 국내...,"[부산, 태어났다, 부산, 대, 재료공학, 공부, 하고, 반도체, 엔지니어, 했다,...","[만, 독자, 사로잡은, 달러, 구트, 꿈, 백화점, 두, 번째, 이야기, 어느덧,...","[프롤로그, 달러, 구트, 다락방, 장, 페니, 첫, 번째, 연봉, 협상, 장, 민...","[내내, 베스트셀러, 위, 독자, 뽑은, 최고, 힐링, 판타지, 잠들어야만, 입장,...",[]
9,10,"[어서, 오세요, 남동, 서점, 입니다, 황, 보름, 장편소설]","[황, 보름]","[클레이, 하우스]",,39,국내도서 소설시희곡 한국소설 한국 장편소설,"[대학, 컴퓨터공학, 전공, 하고, 소프트웨어, 개발자, 했다, 번의, 입사, 퇴사...","[받아들여지고, 있다는, 느낌, 이렇게, 슬프고도, 좋을, 줄, 이야, 브런치, 북...","[서점, 은, 모습, 이어야, 할까, 이제, 더, 는, 울, 지, 않아도, 된다오,...","[먼저, 읽은, 독자, 요청, 종이, 책, 다시, 태어난, 소설, 출, 간, 전자책...","[내, 맘대로, 올해, 책, 모자라지도, 하지도, 않은, 적당한, 온도, 따뜻함과,..."


# 8 단어 정규화

In [15]:
from konlpy.tag import Komoran

In [16]:
def normalize_text(text):
    komoran = Komoran()
    tokens = komoran.morphs(text)
    normalized_tokens = [komoran.normalize(token) for token in tokens]
    normalized_text = ' '.join(normalized_tokens)
    return normalized_text

In [17]:
# 정규화할 열 목록
columns_to_normalize = ['저자소개', '책소개', '출판사서평', '추천사']

In [18]:
# 각 열에 대한 정규화 적용
for column in columns_to_normalize:
    df[column] = df[column].apply(lambda x: normalize_text(x) if isinstance(x, str) else x)

# 최종

In [19]:
df

,순위,도서명,저자,출판사,발행연도,대출횟수,카테고리,저자소개,책소개,목차,출판사서평,추천사
0,1,"[달러, 구트, 꿈, 백화점, 주문, 하신, 꿈, 은, 매진, 입니다, 이미, 장편소설]",[이미],"[팩토리, 나인, 쌤앤, 파커, 스]",,78,국내도서 소설시희곡 장르소설 판타지 국내도서 소설시희곡 한국소설 한국 장편소설 국내...,"[부산, 태어났다, 부산, 대, 재료공학, 공부, 하고, 반도체, 엔지니어, 했다,...","[여기는, 잠들어야만, 입장, 할, 수, 있는, 달러, 구트, 꿈, 백화점, 입니다...","[작가, 말, 프롤로그, 번째, 제자, 유서, 깊은, 가게, 주문, 하신, 꿈, 은...","[텀블벅, 펀딩, 달성, 출, 간, 주, 연속, 종합, 베스트셀러, 위, 독자, 별...",[]
1,2,"[불편한, 편의점, 김호연, 장편소설]",[김호연],"[나무, 옆, 의자]",,78,국내도서 소설시희곡 한국소설 한국 장편소설 국내도서 올해의 책 년 올해의 책 후보...,"[화만, 화, 소설, 넘나, 들며, 온갖, 이야기, 써, 나가는, 전, 천후, 스토...","[최고, 재미, 감동, 선사, 하는, 힐링, 소설, 원조, 주요, 서점, 종합, 베...","[산, 해, 진미, 도시락, 제이, 에스, 오브, 제이, 에스, 삼각김밥, 용도, ...","[원, 플러스, 원, 기쁨, 삼각김밥, 모양, 슬픔, 만, 원, 번의, 폭소, 터지...","[서울역, 홈리스, 지내면서도, 안위, 보다는, 지갑, 잃어버린, 낯선, 부인, 안..."
2,3,"[달빛조각사, 남희성, 게임, 판타지, 소설]",[남희성],"[로크, 미디어]",2014,73,국내도서 소설시희곡 장르소설 판타지,"[어둠의군주, 하이, 마, 태양, 왕, 천년, 마법사, 달빛조각사, 새벽, 여행자,...","[첫, 선, 보인, 온라인과, 오프라인, 넘나, 들며, 다양한, 방식, 대한민국, ...","[다크, 게이머, 탄생, \t\t, 독종, 등장, \t\t, 교관, 의뢰, \t\t...",[],[]
3,4,"[파친코, 이민, 진, 장편소설]",[],"[문학사, 상]",,57,국내도서 소설시희곡 영미소설 영미 장편소설 국내도서 미디어 추천 한겨레 국내도서 ...,"[전, 세계, 뜨거운, 관심, 사랑, 받고, 있는, 한국, 계, 미국인, 소설가, ...","[역사, 는, 버렸지만, 상관없다, 역사, 외면, 당, 한, 재일조선인, 가족, 대...",[],"[회복, 연민, 대한, 강력한, 이야기, 버락, 오바마, 미국, 전, 대통령, 전,...","[올해, 최고, 책, 록산, 게이, 작가, 세기, 견뎌내고, 번영, 이룬, 재일한국..."
4,5,"[듄, 개정판]",[],[황금가지],,55,국내도서 소설시희곡 장르소설 국내도서 소설시희곡 영미소설 영미 장편소설,"[미국, 작가, 가난, 때문, 오레곤, 숙모, 집, 고등학교, 졸업, 한, 허버트,...","[역사상, 가장, 많이, 팔린, 이자, 독자, 뽑은, 역대, 최고, 인, 프랭크, ...","[듄, 부록, 듄, 생태계, 부록, 듄, 종교, 부록, 베네, 게, 세, 리트, 의...","[듄, 견줄, 수, 있는, 건, 반지, 제왕, 외, 에는, 없다, 아서, 클라크, ...","[듄, 견줄, 수, 있는, 건, 반지, 제왕, 외, 에는, 없다, 아서, 클라크, ..."
5,6,"[미드나잇, 라이브러리]",[],"[인, 플루, 엔, 셜]",,54,국내도서 소설시희곡 영미소설 영미 장편소설 국내도서 올해의 책 년 올해의 책 후보...,"[영국, 요크셔, 출신, 동화작가, 겸, 소설가, 다, 출, 간, 한, 첫, 소설,...","[밤, 시, 죽기, 전, 에만, 열리는, 마법, 도서관, 인생, 두, 번째, 기회,...",[],"[영국, 아마존, 종합, 베스트셀러, 위, 미국, 아마존, 뉴욕타임스, 선데이타임스...",[]
6,7,"[지구, 끝, 온실, 김, 초엽, 장편소설]","[김, 초엽]","[자이언트, 북스]",,48,국내도서 소설시희곡 장르소설 국내도서 소설시희곡 한국소설 한국 장편소설 국내도서 ...,"[소설가, 년생, 포스텍, 화학, 전공, 하고, 생화학, 석사, 학위, 받았, 다년...","[김, 초엽, 첫, 장편소설, 간절히, 기다려, 온, 이야기, 이미, 폭넓은, 독자...","[프롤로그, 장, 모스, 바나, 장, 프림, 빌리지, 장, 지구, 끝, 온실, 작가...","[악마, 식물, 내, 정원, 자라고, 있는데, 거, 멸망, 징조, 아니야, 덩굴식물...","[간절히, 원, 해온, 은, 울, 고, 난, 뒤, 에야, 알, 게, 되는, 걸까, ..."
7,8,"[공정하다는, 착각, 능력, 주의, 는, 같은, 기회, 제공, 하는가]",[],"[와이즈, 베리]",,43,국내도서 사회 정치 사회학 사회사사회문화 국내도서 인문 윤리학 국내도서 미디어 추천...,"[이후, 한국, 정의, 열풍, 불러일으켰다, 세, 최연, 소, 하버드, 대학교, 교...","[마이클, 샌델, 만의, 신간, 공정하다는, 착각, 출, 간, 샌델, 기울어진, 사...","[서론, 대학, 입시, 능력, 주의, 입시, 윤리, 능력, 지표, 따내기, 승자, ...","[능력, 주의, 는, 공정하게, 작동, 하는가, 공정함, 정의, 란, 공식, 은, ...","[역시, 마이클, 샌델답다, 에는, 시대, 가장, 예민한, 이슈, 수술, 메스, 대..."
8,9,"[달러, 구트, 꿈, 백화점, 이미, 장편소설, 단골, 손님, 찾습니다]",[이미],"[팩토리, 나인, 쌤앤, 파커, 스]",,42,국내도서 소설시희곡 장르소설 판타지 국내도서 소설시희곡 한국소설 한국 장편소설 국내...,"[부산, 태어났다, 부산, 대, 재료공학, 공부, 하고, 반도체, 엔지니어, 했다,...","[만, 독자, 사로잡은, 달러, 구트, 꿈, 백화점, 두, 번째, 이야기, 어느덧,...","[프롤로그, 달러, 구트, 다락방, 장, 페니, 첫, 번째, 연봉, 협상, 장, 민...","[내내, 베스트셀러, 위, 독자, 뽑은, 최고, 힐링, 판타지, 잠들어야만, 입장,...",[]
9,10,"[어서, 오세요, 남동, 서점, 입니다, 황, 보름, 장편소설]","[황, 보름]","[클레이, 하우스]",,39,국내도서 소설시희곡 한국소설 한국 장편소설,"[대학, 컴퓨터공학, 전공, 하고, 소프트웨어, 개발자, 했다, 번의, 입사, 퇴사...","[받아들여지고, 있다는, 느낌, 이렇게, 슬프고도, 좋을, 줄, 이야, 브런치, 북...","[서점, 은, 모습, 이어야, 할까, 이제, 더, 는, 울, 지, 않아도, 된다오,...","[먼저, 읽은, 독자, 요청, 종이, 책, 다시, 태어난, 소설, 출, 간, 전자책...","[내, 맘대로, 올해, 책, 모자라지도, 하지도, 않은, 적당한, 온도, 따뜻함과,..."


In [20]:
df['서평+추천사'] = df['출판사서평'].fillna('') + df['추천사'].fillna('')
df

,순위,도서명,저자,출판사,발행연도,대출횟수,카테고리,저자소개,책소개,목차,출판사서평,추천사,서평+추천사
0,1,"[달러, 구트, 꿈, 백화점, 주문, 하신, 꿈, 은, 매진, 입니다, 이미, 장편소설]",[이미],"[팩토리, 나인, 쌤앤, 파커, 스]",,78,국내도서 소설시희곡 장르소설 판타지 국내도서 소설시희곡 한국소설 한국 장편소설 국내...,"[부산, 태어났다, 부산, 대, 재료공학, 공부, 하고, 반도체, 엔지니어, 했다,...","[여기는, 잠들어야만, 입장, 할, 수, 있는, 달러, 구트, 꿈, 백화점, 입니다...","[작가, 말, 프롤로그, 번째, 제자, 유서, 깊은, 가게, 주문, 하신, 꿈, 은...","[텀블벅, 펀딩, 달성, 출, 간, 주, 연속, 종합, 베스트셀러, 위, 독자, 별...",[],"[텀블벅, 펀딩, 달성, 출, 간, 주, 연속, 종합, 베스트셀러, 위, 독자, 별..."
1,2,"[불편한, 편의점, 김호연, 장편소설]",[김호연],"[나무, 옆, 의자]",,78,국내도서 소설시희곡 한국소설 한국 장편소설 국내도서 올해의 책 년 올해의 책 후보...,"[화만, 화, 소설, 넘나, 들며, 온갖, 이야기, 써, 나가는, 전, 천후, 스토...","[최고, 재미, 감동, 선사, 하는, 힐링, 소설, 원조, 주요, 서점, 종합, 베...","[산, 해, 진미, 도시락, 제이, 에스, 오브, 제이, 에스, 삼각김밥, 용도, ...","[원, 플러스, 원, 기쁨, 삼각김밥, 모양, 슬픔, 만, 원, 번의, 폭소, 터지...","[서울역, 홈리스, 지내면서도, 안위, 보다는, 지갑, 잃어버린, 낯선, 부인, 안...","[원, 플러스, 원, 기쁨, 삼각김밥, 모양, 슬픔, 만, 원, 번의, 폭소, 터지..."
2,3,"[달빛조각사, 남희성, 게임, 판타지, 소설]",[남희성],"[로크, 미디어]",2014,73,국내도서 소설시희곡 장르소설 판타지,"[어둠의군주, 하이, 마, 태양, 왕, 천년, 마법사, 달빛조각사, 새벽, 여행자,...","[첫, 선, 보인, 온라인과, 오프라인, 넘나, 들며, 다양한, 방식, 대한민국, ...","[다크, 게이머, 탄생, \t\t, 독종, 등장, \t\t, 교관, 의뢰, \t\t...",[],[],[]
3,4,"[파친코, 이민, 진, 장편소설]",[],"[문학사, 상]",,57,국내도서 소설시희곡 영미소설 영미 장편소설 국내도서 미디어 추천 한겨레 국내도서 ...,"[전, 세계, 뜨거운, 관심, 사랑, 받고, 있는, 한국, 계, 미국인, 소설가, ...","[역사, 는, 버렸지만, 상관없다, 역사, 외면, 당, 한, 재일조선인, 가족, 대...",[],"[회복, 연민, 대한, 강력한, 이야기, 버락, 오바마, 미국, 전, 대통령, 전,...","[올해, 최고, 책, 록산, 게이, 작가, 세기, 견뎌내고, 번영, 이룬, 재일한국...","[회복, 연민, 대한, 강력한, 이야기, 버락, 오바마, 미국, 전, 대통령, 전,..."
4,5,"[듄, 개정판]",[],[황금가지],,55,국내도서 소설시희곡 장르소설 국내도서 소설시희곡 영미소설 영미 장편소설,"[미국, 작가, 가난, 때문, 오레곤, 숙모, 집, 고등학교, 졸업, 한, 허버트,...","[역사상, 가장, 많이, 팔린, 이자, 독자, 뽑은, 역대, 최고, 인, 프랭크, ...","[듄, 부록, 듄, 생태계, 부록, 듄, 종교, 부록, 베네, 게, 세, 리트, 의...","[듄, 견줄, 수, 있는, 건, 반지, 제왕, 외, 에는, 없다, 아서, 클라크, ...","[듄, 견줄, 수, 있는, 건, 반지, 제왕, 외, 에는, 없다, 아서, 클라크, ...","[듄, 견줄, 수, 있는, 건, 반지, 제왕, 외, 에는, 없다, 아서, 클라크, ..."
5,6,"[미드나잇, 라이브러리]",[],"[인, 플루, 엔, 셜]",,54,국내도서 소설시희곡 영미소설 영미 장편소설 국내도서 올해의 책 년 올해의 책 후보...,"[영국, 요크셔, 출신, 동화작가, 겸, 소설가, 다, 출, 간, 한, 첫, 소설,...","[밤, 시, 죽기, 전, 에만, 열리는, 마법, 도서관, 인생, 두, 번째, 기회,...",[],"[영국, 아마존, 종합, 베스트셀러, 위, 미국, 아마존, 뉴욕타임스, 선데이타임스...",[],"[영국, 아마존, 종합, 베스트셀러, 위, 미국, 아마존, 뉴욕타임스, 선데이타임스..."
6,7,"[지구, 끝, 온실, 김, 초엽, 장편소설]","[김, 초엽]","[자이언트, 북스]",,48,국내도서 소설시희곡 장르소설 국내도서 소설시희곡 한국소설 한국 장편소설 국내도서 ...,"[소설가, 년생, 포스텍, 화학, 전공, 하고, 생화학, 석사, 학위, 받았, 다년...","[김, 초엽, 첫, 장편소설, 간절히, 기다려, 온, 이야기, 이미, 폭넓은, 독자...","[프롤로그, 장, 모스, 바나, 장, 프림, 빌리지, 장, 지구, 끝, 온실, 작가...","[악마, 식물, 내, 정원, 자라고, 있는데, 거, 멸망, 징조, 아니야, 덩굴식물...","[간절히, 원, 해온, 은, 울, 고, 난, 뒤, 에야, 알, 게, 되는, 걸까, ...","[악마, 식물, 내, 정원, 자라고, 있는데, 거, 멸망, 징조, 아니야, 덩굴식물..."
7,8,"[공정하다는, 착각, 능력, 주의, 는, 같은, 기회, 제공, 하는가]",[],"[와이즈, 베리]",,43,국내도서 사회 정치 사회학 사회사사회문화 국내도서 인문 윤리학 국내도서 미디어 추천...,"[이후, 한국, 정의, 열풍, 불러일으켰다, 세, 최연, 소, 하버드, 대학교, 교...","[마이클, 샌델, 만의, 신간, 공정하다는, 착각, 출, 간, 샌델, 기울어진, 사...","[서론, 대학, 입시, 능력, 주의, 입시, 윤리, 능력, 지표, 따내기, 승자, ...","[능력, 주의, 는, 공정하게, 작동, 하는가, 공정함, 정의, 란, 공식, 은, ...","[역시, 마이클, 샌델답다, 에는, 시대, 가장, 예민한, 이슈, 수술, 메스, 대...","[능력, 주의, 는, 공정하게, 작동, 하는가, 공정함, 정의, 란, 공식, 은, ..."
8,9,"[달러, 구트, 꿈, 백화점, 이미, 장편소설, 단골, 손님, 찾습니다]",[이미],"[팩토리, 나인, 쌤앤, 파커, 스]",,42,국내도서 소설시희곡 장르소설 판타지 국내도서 소설시희곡 한국소설 한국 장편소설 국내...,"[부산, 태어났다, 부산, 대, 재료공학, 공부, 하고, 반도체, 엔지니어, 했다,...","[만, 독자, 사로잡은, 달러, 구트, 꿈, 백화점, 두, 번째, 이야기, 어느덧,...","[프롤로그, 달러, 구트, 다락방, 장, 페니, 첫, 번째, 연봉, 협상, 장, 민...","[내내, 베스트셀러, 위, 독자, 뽑은, 최고, 힐링, 판타지, 잠들어야만, 입장,...",[],"[내내, 베스트셀러, 위, 독자, 뽑은, 최고, 힐링, 판타지, 잠들어야만, 입장,..."
9,10,"[어서, 오세요, 남동, 서점, 입니다, 황, 보름, 장편소설]","[황, 보름]","[클레이, 하우스]",,39,국내도서 소설시희곡 한국소설 한국 장편소설,"[대학, 컴퓨터공학, 전공, 하고, 소프트웨어, 개발자, 했다, 번의, 입사, 퇴사...","[받아들여지고, 있다는, 느낌, 이렇게, 슬프고도, 좋을, 줄, 이야, 브런치, 북...","[서점, 은, 모습, 이어야, 할까, 이제, 더, 는, 울, 지, 않아도, 된다오,...","[먼저, 읽은, 독자, 요청, 종이, 책, 다시, 태어난, 소설, 출, 간, 전자책...","[내, 맘대로, 올해, 책, 모자라지도, 하지도, 않은, 적당한, 온도, 따뜻함과,...","[먼저, 읽은, 독자, 요청, 종이, 책, 다시, 태어난, 소설, 출, 간, 전자책..."


In [21]:
df.to_excel('경기대 전처리.xlsx',index=False)

# 9 토픽 모델링 LDA

- 일관성 점수가 높은 모델은 주제 간의 일관성이 높아 의미 있는 토픽을 잘 나타내는 경향